In [73]:
from pydantic import BaseModel, field_validator, ValidationError
import datetime
import pandas as pd
import itertools

2. Валидировать входящий JSON используя модель pydantic (из ТЗ известно что поле "key1" имеет тип int, "key2"(datetime), "key3"(str))

In [74]:
a = {
    "Columns": [1, datetime.datetime(2024, 5, 17), 'key3'],
    "Description": ['Банковские API каких-то важных документов'],
    "RowCount": 2,
    "Rows":[
        ["value1", "value2", "value3"],
        ["value4", "value5", "value6"]
    ]
}

In [75]:
class check(BaseModel):
    Columns: list
    Description: list
    RowCount: int
    Rows: list[list]

    @field_validator('Columns')
    def elements_in_columns(cls, columns):
        if len(columns) < 3:
            raise ValueError('List must contain at least three elements')

        if not isinstance(columns[0], int):
            raise ValueError('Первый элемент должен быть типа int')
        
        if not isinstance(columns[1], datetime.datetime):
            raise ValueError('Второй элемент должен быть типа datetime')
        
        if not isinstance(columns[2], str):
            raise ValueError('Третий элемент должен быть типа str')

        return columns


In [76]:
try:
    candidate = check(**a)
    print(candidate.model_dump())
except ValidationError as err:
    print('Error validating candidate info: ', err)

{'Columns': [1, datetime.datetime(2024, 5, 17, 0, 0), 'key3'], 'Description': ['Банковские API каких-то важных документов'], 'RowCount': 2, 'Rows': [['value1', 'value2', 'value3'], ['value4', 'value5', 'value6']]}


3. Представить данные "Columns" и "Rows" в виде плоского csv-подобного pandas.DataFrame 

In [77]:
df = pd.DataFrame()
l = []
for i in a:
    match i:
        case "Rows":
            df = pd.DataFrame(a[i], columns=l)
        case "Columns":
            l.append(a[i])
            l = list(itertools.chain(*l))

In [78]:
df

,1,2024-05-17 00:00:00,key3
0,value1,value2,value3
1,value4,value5,value6


4. В полученном DataFrame произвести переименование полей по след. маппингу key1" -> "document_id", "key2" -> "document_dt", "key3" -> "document_name"

In [79]:
df = df.rename(columns={df.columns[0]: "document_id", df.columns[1]: "document_dt", df.columns[2]: "document_name"})

In [80]:
df['load_dt'] =[datetime.datetime.now() for i in df['document_id']]

In [81]:
df

,document_id,document_dt,document_name,load_dt
0,value1,value2,value3,2024-05-17 14:36:16.822248
1,value4,value5,value6,2024-05-17 14:36:16.822253
